In [1]:
import numpy as np
import seal
from seal import EncryptionParameters, SEALContext, KeyGenerator, Evaluator, Encryptor, Decryptor, FractionalEncoder, Ciphertext, Plaintext

In [2]:
# model
w = [0.1, 0.05, 0.05, 0.2, 0.05, 0.3, 0.1, 0.025, 0.075, 0.05]
# data (sensitive)
x = [3.1, 4.159, 2.65, 3.5897, 9.3, 2.3, 8.46, 2.64, 3.383, 2.795]

In [3]:
N = len(x)

In [4]:
np.sum(np.array(w)*np.array(x))

3.828865

In [5]:
parms = EncryptionParameters()

In [6]:
parms.set_poly_modulus("1x^2048 + 1")
parms.set_coeff_modulus(seal.coeff_modulus_128(2048))
parms.set_plain_modulus(1 << 8)

In [7]:
context = SEALContext(parms)

In [8]:
evaluator = Evaluator(context)

In [9]:
keygen = KeyGenerator(context)

In [10]:
public_key = keygen.public_key()
encryptor = Encryptor(context, public_key)

In [11]:
encoder = FractionalEncoder(context.plain_modulus(), context.poly_modulus(), 64, 32, 3)

In [12]:
# encrypt the data
encrypted_x = []
for i in range(N):
    encrypted_x.append(Ciphertext())
    encryptor.encrypt(encoder.encode(x[i]), encrypted_x[i])

In [13]:
# no need to encrypt the model
encoded_w = []
for i in range(N):
    encoded_w.append(encoder.encode(w[i]))

In [14]:
# dot product
for i in range(N):
    evaluator.multiply_plain(encrypted_x[i], encoded_w[i])
encrypted_result = Ciphertext()
evaluator.add_many(encrypted_x, encrypted_result)

In [15]:
encrypted_result

In [16]:
private_key = keygen.secret_key()

In [17]:
decryptor = Decryptor(context, private_key)

In [18]:
plain_result = Plaintext()
decryptor.decrypt(encrypted_result, plain_result)
result = encoder.decode(plain_result)

In [19]:
result

3.8288649999999986

In [20]:
np.sum(np.array(w)*np.array(x))

3.828865

In [21]:
result-np.sum(np.array(w)*np.array(x))

-1.3322676295501878e-15